In [ ]:
# 
# This Notebook requires the following environment variables (.env file):
# 

# PACKAGE_LOCAL_WORKING_DIR=
# PACKAGE_INPUT_CONTAINER_SAS_URI=
# PACKAGE_OUTPUT_CONTAINER_SAS_URI=


In [ ]:
import os
import uuid
import zipfile
from azure.storage.blob import ContainerClient

from dotenv import load_dotenv
load_dotenv()

PACKAGE_NAME = str(uuid.uuid4())
LOCAL_WORKING_DIR = os.getenv("PACKAGE_LOCAL_WORKING_DIR")
INPUT_CONTAINER_SAS_URI = os.getenv("PACKAGE_INPUT_CONTAINER_SAS_URI")
OUTPUT_CONTAINER_SAS_URI = os.getenv("PACKAGE_OUTPUT_CONTAINER_SAS_URI")

def get_container_client_from_sas(sas_uri):
    """Get a container client directly from a container SAS URI."""
    return ContainerClient.from_container_url(sas_uri)

def download_filtered_audio_blobs(container_client, file_extensions=('.mp3', '.wav'), max_duration=15, local_folder="downloads"):
    """Download filtered audio blobs from a container"""
    os.makedirs(local_folder, exist_ok=True)
    
    downloaded_files = []
    blob_list = list(container_client.list_blobs())

    print(f"Found {len(blob_list)} total blobs in container")

    for blob in blob_list:
        #  filter by file extension
        if not blob.name.lower().endswith(file_extensions):
            print(f"Skipping {blob.name} - unsupported file extension")
            continue

        blob_client = container_client.get_blob_client(blob.name)
        props = blob_client.get_blob_properties()
        metadata = props.metadata

        # Filter by duration in metadata
        try:
            duration = float(metadata.get('duration', 0))        
        except ValueError:
            print(f"Skipping {blob.name} - invalid duration metadata")
            continue

        if duration >= max_duration:
            print(f"Skipping {blob.name}: duration {duration}s >= {max_duration}s")
            continue

        # Filter by presence of text metadata
        text = metadata.get('text', '')

        if not text:
            print(f"Skipping {blob.name} - no text metadata found")
            continue
    
        local_file_path = os.path.join(local_folder, os.path.basename(blob.name))
        print(f"Downloading {blob.name} to {local_file_path}")

        with open(local_file_path, "wb") as file:
            blob_data = blob_client.download_blob()
            file.write(blob_data.readall())

        downloaded_files.append({
            "blob_name": blob.name,
            "local_path": local_file_path,
            "duration": duration,
            "text": text
        })

    print(f"Downloaded {len(downloaded_files)} files that match the criteria")
    return downloaded_files

def create_package_manifest(package_name, audio_files, output_folder):
    """Create a package manifest file."""
    manifest_path = os.path.join(output_folder, f"{package_name}.txt")
    
    with open(manifest_path, 'w') as manifest_file:
        for item in audio_files:
            filename_wo_ext = os.path.splitext(os.path.basename(item['local_path']))[0]
            manifest_file.write(f"{filename_wo_ext}\t{item['text']}\n")
    
    print(f"Manifest created at {manifest_path}")
    return manifest_path

def create_zip_from_files(audio_files, zip_path):
    """Create a ZIP archive containing all downloaded audio files."""
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for item in audio_files:
            arcname = os.path.basename(item['local_path'])
            zipf.write(item['local_path'], arcname=arcname)
    print(f"Created ZIP archive at {zip_path}")
    return zip_path

def upload_files(container_client, local_file_paths, folder=None):
    """Upload multiple files to the specified container."""

    uploaded_files = []

    for local_file_path in local_file_paths:
        file_name = os.path.basename(local_file_path)
        if folder:
            blob_name = f"{folder}/{file_name}"
        else:
            blob_name = file_name 

        print(f"Uploading {local_file_path}")
        with open(local_file_path, "rb") as data:
            container_client.upload_blob(
                name=blob_name, 
                data=data, 
                overwrite=True
            )

        uploaded_files.append(blob_name)

    return uploaded_files
#
# Main
#

print("Starting packaging process for package: ", PACKAGE_NAME)

input_container_client = get_container_client_from_sas(INPUT_CONTAINER_SAS_URI)
output_container_client = get_container_client_from_sas(OUTPUT_CONTAINER_SAS_URI)
package_working_dir = os.path.join(LOCAL_WORKING_DIR, PACKAGE_NAME)

os.makedirs(LOCAL_WORKING_DIR, exist_ok=True)
os.makedirs(package_working_dir, exist_ok=True)

print("Downloading audio files from input container...")
audio_files = download_filtered_audio_blobs(
    input_container_client,
    file_extensions=('.mp3', '.wav'),
    max_duration=15,
    local_folder=package_working_dir
)
print(f"Downloaded {len(audio_files)} audio files.")

print("Creating package manifest...")
manifest_path = create_package_manifest(
    PACKAGE_NAME,
    audio_files,
    LOCAL_WORKING_DIR
)
print(f"Manifest created at {manifest_path}")

print("Creating zip package...")
zip_path = create_zip_from_files(
    audio_files, 
    os.path.join(LOCAL_WORKING_DIR, f"{PACKAGE_NAME}.zip")
)
print(f"Zip package created at {zip_path}")

print("Uploading files to output container...")
uploaded_files = upload_files(output_container_client, [manifest_path, zip_path])
print(f"Uploaded {len(uploaded_files)}.")

print("Done")
